In [20]:
import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from torchvision.models import vgg19

In [2]:
class ResidualBlock(nn.Module):
    def __init__(
            self,
            in_channels = 64,
            out_channels = 64,
            stride = 1,
            kernel_size = 3,
            padding = 1
    ):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = in_channels,
                               out_channels = out_channels,
                               kernel_size = kernel_size,
                               stride = stride,
                               padding= padding,
                               bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.activation = nn.PReLU(num_parameters=out_channels)

        self.conv2 = nn.Conv2d(in_channels = in_channels,
                               out_channels = out_channels,
                               kernel_size = kernel_size,
                               stride = stride,
                               padding= padding,
                               bias=False)

        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = out + x
        return out


In [10]:
class GeneratorUpsample(nn.Module):
    def __init__(
            self,
            in_channels = 64,
            upsample_value = 2,
            stride = 1,
            kernel_size = 3,
            padding = 1
    ):
        super().__init__()
        self.upsample_value = upsample_value
        self.in_channels = in_channels
        self.stride = stride
        self.kernel_size = kernel_size
        self.padding = padding

        self.conv = nn.Conv2d(in_channels=self.in_channels,
                              out_channels=self.in_channels*upsample_value*upsample_value,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              stride = self.stride)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor=upsample_value)
        self.activation = nn.PReLU(in_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        x = self.activation(x)
        return x

In [35]:
class Generator(nn.Module):
    def __init__(
            self,
            in_channels = 3,
            out_channels = 3,
            hidden_channels = 64,
            num_res_blocks = 16
    ):
        super().__init__()

        self.in_channels = in_channels
        self.num_res_blocks = num_res_blocks
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels

        self.initial_conv = nn.Conv2d(
            in_channels = self.in_channels,
            out_channels = self.hidden_channels,
            stride = 1,
            kernel_size = 9,
            padding = 4
        )
        self.activation = nn.PReLU(self.hidden_channels)
        self.residual_blocks = nn.Sequential(*[ResidualBlock() for _ in range(num_res_blocks)])
        self.middle_conv = nn.Conv2d(
            in_channels = self.hidden_channels,
            out_channels = self.hidden_channels,
            stride = 1,
            kernel_size = 3,
            padding = 1
        )
        self.bn = nn.BatchNorm2d(self.hidden_channels)
        self.final_conv = nn.Conv2d(
            in_channels = self.hidden_channels,
            out_channels = self.out_channels,
            stride = 1,
            kernel_size = 9,
            padding = 4
        )
        self.up1 = GeneratorUpsample()
        self.up2 = GeneratorUpsample()


    def forward(self, x):
        x = self.initial_conv(x)
        x = self.activation(x)
        out = self.residual_blocks(x)
        out = self.middle_conv(out)
        out = self.bn(out)
        out = out + x
        out = self.up1(out)
        out = self.up2(out)
        out = self.final_conv(out)
        return out

In [38]:
class ConvBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,
            stride,
            kernel_size=3,
            padding=1
    ):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(
                in_channels = in_channels,
                out_channels = out_channels,
                stride = stride,
                kernel_size = kernel_size,
                padding = padding
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    def forward(self, x):
        return self.net(x)

In [48]:
class Discriminator(nn.Module):
    def __init__(
            self,
            in_channels = 3
    ):
        super().__init__()
        self.in_channels = in_channels
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=self.in_channels, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            ConvBlock(in_channels=64, out_channels=64, stride=2),
            ConvBlock(in_channels=64, out_channels=128, stride=1),
            ConvBlock(in_channels=128, out_channels=128, stride=2),
            ConvBlock(in_channels=128, out_channels=256, stride=1),
            ConvBlock(in_channels=256, out_channels=256, stride=2),
            ConvBlock(in_channels=256, out_channels=512, stride=1),
            ConvBlock(in_channels=512, out_channels=512, stride=2),
            nn.Flatten(),
            nn.Linear(in_features=18432, out_features=1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

In [14]:
class MyDataset(Dataset):
    def __init__(
            self,
            folder_path,
            train = True,
            hr_res = 96
    ):
        super().__init__()
        image_names = os.listdir(path=folder_path)
        self.image_paths = [os.path.join(folder_path, x) for x in image_names]
        self.hr_res = hr_res
        self.lr_res = hr_res//4
        self.initial_transform = A.Compose(
            [
                A.RandomCrop(self.hr_res, self.hr_res),
                A.HorizontalFlip(p=0.5) if train else None,
                A.RandomRotate90(p=0.5) if train else None
            ]
        )
        self.hr_transform = A.Compose(
            [
                A.Normalize(mean=(0, 0, 0), std=(1, 1, 1)),
                ToTensorV2()
            ]
        )
        self.lr_transform = A.Compose(
            [
                A.Resize(self.lr_res, self.lr_res),
                A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
                ToTensorV2()
            ]
        )

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.initial_transform(image=image)['image']
        hr_image = self.hr_transform(image=image)['image']
        lr_image = self.lr_transform(image=image)['image']
        return hr_image, lr_image

In [15]:
dataset = MyDataset('finished/train/dataraw/hires')

In [19]:
next(iter(dataset))[1].shape

torch.Size([3, 24, 24])

In [22]:
check = vgg19()

In [23]:
check

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [6]:
os.path.join('finished/train/dataraw/hires', os.listdir('finished/train/dataraw/hires')[0])

'finished/train/dataraw/hires\\dt_train_1.jpg'

In [4]:
os.listdir('finished/train/dataraw/hires')

['dt_train_1.jpg',
 'dt_train_10.jpg',
 'dt_train_100.jpg',
 'dt_train_1000.jpg',
 'dt_train_1001.jpg',
 'dt_train_1002.jpg',
 'dt_train_1003.jpg',
 'dt_train_1004.jpg',
 'dt_train_1005.jpg',
 'dt_train_1006.jpg',
 'dt_train_1007.jpg',
 'dt_train_1008.jpg',
 'dt_train_1009.jpg',
 'dt_train_101.jpg',
 'dt_train_1010.jpg',
 'dt_train_1011.jpg',
 'dt_train_1012.jpg',
 'dt_train_1013.jpg',
 'dt_train_1014.jpg',
 'dt_train_1015.jpg',
 'dt_train_1016.jpg',
 'dt_train_1017.jpg',
 'dt_train_1018.jpg',
 'dt_train_1019.jpg',
 'dt_train_102.jpg',
 'dt_train_1020.jpg',
 'dt_train_1021.jpg',
 'dt_train_1022.jpg',
 'dt_train_1023.jpg',
 'dt_train_1024.jpg',
 'dt_train_1025.jpg',
 'dt_train_1026.jpg',
 'dt_train_1027.jpg',
 'dt_train_1028.jpg',
 'dt_train_1029.jpg',
 'dt_train_103.jpg',
 'dt_train_1030.jpg',
 'dt_train_1031.jpg',
 'dt_train_1032.jpg',
 'dt_train_1033.jpg',
 'dt_train_1034.jpg',
 'dt_train_1035.jpg',
 'dt_train_1036.jpg',
 'dt_train_1037.jpg',
 'dt_train_1038.jpg',
 'dt_train_1039.jpg

In [49]:
Discriminator()(torch.ones(10, 3, 96, 96).float()).shape

torch.Size([10, 1])

In [36]:
gen = Generator()

In [37]:
gen(torch.ones(10, 3, 128, 128)).shape

torch.Size([10, 64, 128, 128])
torch.Size([10, 64, 512, 512])


torch.Size([10, 3, 512, 512])

In [18]:
foo = lambda x, y: x*y
foo(*[2,2])

4

In [15]:
*[ResidualBlock() for x in range(16)]

SyntaxError: can't use starred expression here (3322405202.py, line 1)

In [11]:
up = GeneratorUpsample()

In [12]:
up(torch.ones(10, 64, 128, 128)).shape

torch.Size([10, 64, 256, 256])

In [8]:
ttt = ResidualBlock()

In [4]:
ttt(torch.ones(10, 64, 128, 128)).shape

torch.Size([10, 64, 128, 128])

In [4]:
class ConvBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,

    ):


NameError: name 'self' is not defined

In [2]:
torch.cuda.is_available()

True